# Generating Shapes

You can build a rect yourself, but for more complex shapes you benefit from having generators which handle the hard parts for you.

This module is also the API design basis for the geo stuff, so it's good to understand this stuff before trying to jump into geo modules.

In [2]:
%load_ext py_d3

## Arcs

Arcs are pie-like thingies. As with histograms, there's a `d3.arc` method which emits a modifiable factory constructor. Here's a basic build:

In [5]:
%%d3 4.4.4

<script>
let arc = d3.arc()({
  innerRadius: 0,
  outerRadius: 100,
  startAngle: 0,
  endAngle: Math.PI / 2
});
console.log(arc); // emits M6.123233995736766e-15,-100A100,100,0,0,1,100,0L0,0Z
</script>

Ok, let's add that to an SVG. Notice that the arc is generated as a path SVG element with a datum, which we can attach to the display thusly.

In [11]:
%%d3 4.4.4

<svg style="width:200px; height:200px;"></svg>

<script>
let arc = d3.arc()({
  innerRadius: 0,
  outerRadius: 100,
  startAngle: 0,
  endAngle: Math.PI / 2
});

d3.select("svg").append("path").attr("d", arc);
</script>

The default coordinate is (0,0), so the arc is above the display here :) You have to adjust it into place yourself.

In [13]:
%%d3 4.4.4

<svg style="width:200px; height:200px;"></svg>

<script>
let arc = d3.arc()({
  innerRadius: 0,
  outerRadius: 100,
  startAngle: 0,
  endAngle: Math.PI / 2
});

d3.select("svg").append("path").attr("d", arc).attr("transform", "translate(100, 100)")
</script>

Great! We can play around with the options a bit, but they're all obvious.

In [18]:
%%d3 4.4.4

<svg style="width:200px; height:200px;"></svg>

<script>
let arc = d3.arc()({
  innerRadius: 75,
  outerRadius: 100,
  startAngle: Math.PI/4,
  endAngle: Math.PI*2
});

d3.select("svg").append("path").attr("d", arc).attr("transform", "translate(100, 100)")
</script>

All right, let's look at some other arguments. First of all, `centroid` computes the arc center, for ease of labeling.

In [30]:
%%d3 4.4.4

<svg style="width:200px; height:200px;"></svg>

<script>
let arc_segment_dict = {
  innerRadius: 60,
  outerRadius: 100,
  startAngle: Math.PI/4,
  endAngle: Math.PI*1
}

let segment = d3.arc()(arc_segment_dict);
let centroid = d3.arc().centroid(arc_segment_dict);
console.log(centroid);

d3.select("svg").append("path").attr("d", segment).attr("transform", "translate(100, 100)").attr("fill", "steelblue");
d3.select("svg").append("circle")
    .attr("r", "5").attr("cx", centroid[0]).attr("cy", centroid[1])
    .attr("transform", "translate(100, 100)").attr("fill", "white");
</script>

The attributes in the init can also be specified before generation via argument. Let's use them here.

In [49]:
%%d3 4.4.4

<svg style="width:200px; height:200px;"></svg>

<script>
// define the data
let start_angles = [0, 60, 75, 80, 120, 200, 270, 360];
let end_angles = start_angles.slice(-1).concat(start_angles.slice(0, -1));
let colors = ['steelblue', 'green', 'gray', 'brown', 'light red', 'purple', 'yellow', 'pink']


let arc_segment_dict = {
    innerRadius: 60,
    outerRadius: 100,
    startAngle: Math.PI/4,
    endAngle: Math.PI*1,
}

for(let i=0; i < start_angles.length; i++) {
    stuff = arc_segment_dict;
    stuff['startAngle'] = start_angles[i] / 180 * Math.PI;
    stuff['endAngle'] = end_angles[i] / 180 * Math.PI;
    let segment = d3.arc()(stuff);
    let color = colors[i];
    d3.select("svg").append("path")
        .attr("d", segment).attr("transform", "translate(100, 100)").attr("fill", color);
}
</script>

## Pie

The pie generators don't produce a shape, they produce something that can be fed to `arc`.

In [52]:
%%d3 4.4.4

<script>
// define the data
let data = [1, 1, 2, 3, 5, 8, 13, 21];
let arcs = d3.pie()(data);

console.log(arcs);
</script>

The default generator generates an array of objects with the following properties (example):

    data:8
    endAngle:4.886921905584122
    index:2
    padAngle:0
    startAngle:3.9560796378538132
    value:8

Then, feeding that into arc:

In [84]:
%%d3 4.4.4

<svg style="width:200px; height:200px;"></svg>

<script>
// define the data
let data = [0, 60, 75, 80, 120, 200, 270, 360];
let arc_data = d3.pie()(data);

let arc = d3.arc().innerRadius(25).outerRadius(50).padAngle(d => 0.1);

let path = d3.select("svg").selectAll('path')
  .data(arc_data)
  .enter()
  .append('path')
  .attr('d', arc)
  .attr('fill', 'steelblue').attr("transform", "translate(100, 100)");
</script>

In [62]:
%%d3 4.4.4

<svg style="width:200px; height:200px;"></svg>

<script>
// define the data
let data = [0, 60, 75, 80, 120, 200, 270, 360];
let arc_data = d3.pie()(data);

let arc = d3.arc().innerRadius(25).outerRadius(50).cornerRadius(5);
let pie = d3.pie();

let path = d3.select("svg").selectAll('path')
  .data(arc_data)
  .enter()
  .append('path')
  .attr('d', arc)
  .attr('fill', 'steelblue').attr("transform", "translate(100, 100)");
</script>

In [69]:
%%d3 4.4.4

<svg style="width:200px; height:200px;"></svg>

<script>
// define the data
let data = [0, 60, 75, 80, 120, 200, 270, 360];
let arc_data = d3.pie()(data);

let arc = d3.arc().innerRadius(25).outerRadius(50).padAngle(0.1); // dont use directly! Bias!
// cf. https://bl.ocks.org/mbostock/053fcc2295a445afab07
let pie = d3.pie();

let path = d3.select("svg").selectAll('path')
  .data(arc_data)
  .enter()
  .append('path')
  .attr('d', arc)
  .attr('fill', 'steelblue').attr("transform", "translate(100, 100)");
</script>

Value accessor:

In [71]:
%%d3 4.4.4

<svg style="width:200px; height:200px;"></svg>

<script>
// define the data
var data = [
  {"number":  4, "name": "Locke"},
  {"number":  8, "name": "Reyes"},
  {"number": 15, "name": "Ford"},
  {"number": 16, "name": "Jarrah"},
  {"number": 23, "name": "Shephard"},
  {"number": 42, "name": "Kwon"}
];

let arc_data = d3.pie().value(d => d.number)(data);

let arc = d3.arc().innerRadius(25).outerRadius(50).padAngle(0.1);
let pie = d3.pie();

let path = d3.select("svg").selectAll('path')
  .data(arc_data)
  .enter()
  .append('path')
  .attr('d', arc)
  .attr('fill', 'steelblue').attr("transform", "translate(100, 100)");
</script>

There's also sorting, and a few other things I'm leaving out.

## Line

Not sure yet what the diff between "Line" and "Curve" is.

In [81]:
%%d3 4.4.4

<svg style="width:200px; height:200px;"></svg>

<script>
// define the data
let data = [
    [0, 50],
    [10, 100],
    [20, 90],
    [30, 50],
    [40, 0],
    [50, 10],
    [60, 20],
    [70, 70],
    [80, 70],
    [90, 90],
    [100, 100]
];

let line = d3.line()(data);

d3.select("svg").append("path").attr("d", line).attr("stroke", "black").attr("fill", "white")
    .attr("transform", "translate(100, 100)")
</script>